In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

from datetime import datetime
import json

from os import listdir, getenv

from sentence_transformers import SentenceTransformer

In [ ]:
hf_token = getenv("HF_TOKEN")

In [ ]:
model_id = "microsoft/Phi-3-mini-4k-instruct"

# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

In [ ]:
messages = [ 
    {"role": "system", "content": "You are a helpful AI assistant."}, 
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"}, 
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."}, 
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"}, 
] 

pipe = pipeline( 
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
) 

generation_args = { 
    "max_new_tokens": 500, 
    "return_full_text": False, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

output = pipe(messages, **generation_args) 
print(output[0]['generated_text']) 

In [ ]:
model_name = "microsoft/Phi-3-mini-4k-instruct"
# model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_name = "gpt2-large"

model = HookedTransformer.from_pretrained(model_name=model_name, device=device)

In [ ]:
prompt = """
Write a short blog post about a recipe and the inspiration behind it.
 Do not include a title.
 Only reveal the dish after the story.
 Start with a one, two or three paragraph story and then move to the recipe.
 To re-iterate, do not include a title.
""".replace(
    "\n", ""
)

start = """
The recipe I’m sharing today is an amalgamation of my grandmother's traditional Italian pasta dish and my
 recent trip to Italy. 
<br><br>
My grandmother, Nonna Maria, always made the most delicious pasta. We would gather around the table,
 eagerly awaiting the rich aroma that filled our kitchen every time she prepared her signature dish.
 The scent would always linger in the air, as if it was a promise of the warmth and love that accompanied
 each bite. 
<br><br>
When I travelled to Italy, I was determined to recreate the dish to the best of my abilities. I spent hours
 exploring the bustling markets, indulging in the local cuisine, and soaking up the vibrant culture.
 Every meal was an experience, and I was eager to incorporate the flavors and techniques I learned into
 my recipe.
<br><br>
After returning home, I sat down to deconstruct the dish and rebuild it in my kitchen. I started with
 the pasta, carefully selecting the perfect type of wheat and rolling it out to the perfect thickness.
 The sauce was a blend of traditional Italian ingredients, infused with my unique twist. The result was
 a dish that not only brought back cherished memories but also embodied the spirit of my journey through
 Italy.
<br><br>
""".replace("\n","").replace("<br>", "\n")

print(prompt)
print('='*10)
print(start)

In [ ]:
temperature = 0.7

for _ in range(5):
    output = model.generate(
        input=prompt + "\n\n" + start,
        max_new_tokens=150,
        temperature=temperature,
        prepend_bos=False,
    )

    # export output to json file, also tracking choices made
    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    path_to_export = f"../results/{current_time}.json"

    # Create a dictionary with the data to be written to the JSON file
    data = {
        "model": model_name,
        "temperature": temperature,
        "prompt": prompt,
        "start": start,
        "output": output,
    }

    # Open the file in write mode and write the data as JSON
    with open(path_to_export, "w") as file:
        json.dump(data, file, indent=4)

In [ ]:
print(output)

## Load results from JSON files

In [ ]:
# load results from json files
files = listdir("../results")
files = [file for file in files if file.startswith("2024-07-10")]

for file in files:
    print(file)

In [ ]:
results = []

for file in files:
    with open(f"../results/{file}", "r") as f:
        results.append(json.load(f))

results[0]

In [ ]:
outputs = [r["output"][len(r["prompt"])+len(r["start"])+2:] for r in results]
outputs[0]

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

# Calculate embeddings by calling model.encode()
embeddings = model.encode(outputs)
print(embeddings.shape)

# Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)